# Analysis for a 3 quantum damping channel discrimination

In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from typing import cast, Tuple, List
from qcd import load_object_from_file
from qcd.typings.configurations import OptimalConfigurations
from qcd.configurations import OneShotConfiguration
import math

In [2]:
results = load_object_from_file(name='20210414b_C2_A2_1000_10000_3_1_20', path='../demo/data/')

In [ ]:
class OptimalConfigurations(TypedDict, total=False):
    eta_groups: List[List[float]]
    best_algorithm: List[str]
    probabilities: List[float]
    configurations: List[ChannelConfiguration]
    number_calls_made: List[int]
    legacy: bool

return {'state_probability': self._state_probability if hasattr(self, 'state_probability') else self._theta,
'angle_rx': self._angle_rx,
'angle_ry': self._angle_ry,
'eta_group': self._eta_group,
}

In [3]:
def get_number_eta_pairs(eta_groups)->int:
    eta_third = eta_groups[0][2]
    number_eta_pairs = 0
    for eta_group in eta_groups:
        if eta_group[2] != eta_third:
            return number_eta_pairs
        number_eta_pairs += 1

In [115]:
def get_etas_and_probabilities_from_optimal_results(optimal_results: OptimalConfigurations)-> List[dict]:
    """ From the optimal results, obtain the etas and the result probabilities for each 3-eta execution.
        Return a list of etas used (in degrees) and the probabilities with the associated eta pair.
    """
    probabilities = []
    eta_pairs = []
    etas_third_channel = []
    
    for configuration in optimal_results['configurations']:
        eta_pairs.append((int(math.degrees(cast(OneShotConfiguration, configuration).eta_group[0])),
                         int(math.degrees(cast(OneShotConfiguration, configuration).eta_group[1]))))
        etas_third_channel.append(int(math.degrees(cast(OneShotConfiguration, configuration).eta_group[2])))
        probabilities.append(cast(OneShotConfiguration, configuration).state_probability)

    number_eta_pairs = get_number_eta_pairs(eta_groups=optimal_results['eta_groups'])
    number_third_channels = int(len(optimal_results['eta_groups']) / number_eta_pairs)

    results_to_plot = [{'probabilities': [],'etas_third_channel': [],'eta_pair':(0,0)} for i in range(number_eta_pairs)]
    
    for idx, (probability, eta_third, eta_pair) in enumerate(zip(probabilities, etas_third_channel, eta_pairs)):
        results_to_plot[idx % number_eta_pairs]['probabilities'].append(probability)
        results_to_plot[idx % number_eta_pairs]['etas_third_channel'].append(eta_third)
        results_to_plot[idx % number_eta_pairs]['eta_pair'] = eta_pair
    return results_to_plot
    

In [116]:
parsed_results = get_etas_and_probabilities_from_optimal_results(optimal_results=results)

In [117]:
len(parsed_results)

6

In [118]:
parsed_results

[{'probabilities': [0.8333333333333333,
   0.8337905807041609,
   0.8333333333333333,
   0.8333333333333333,
   0.8333333333333333,
   0.7222222222222221,
   0.8333332636416139,
   0.8333333333333333,
   0.5,
   0.796245491032871,
   0.8331809175430573,
   0.8209876543209875,
   0.8333333333333333,
   0.7222222222222221,
   0.7962962937151215,
   0.7962962962962963,
   0.8333333333333333,
   0.7962962962962963,
   0.7222222222222221,
   0.7222222222222221],
  'etas_third_channel': [0,
   4,
   9,
   14,
   18,
   23,
   28,
   33,
   37,
   42,
   47,
   52,
   56,
   61,
   66,
   71,
   75,
   80,
   85,
   90],
  'eta_pair': (0, 29)},
 {'probabilities': [0.8333333333333333,
   0.8333333333333333,
   0.7962962962962963,
   0.7962962962962963,
   0.8333333333333333,
   0.8333333333333333,
   0.8333331242581752,
   0.796296296009499,
   0.7962962962962963,
   0.8333333333333333,
   0.7962962962962963,
   0.8333333333333333,
   0.8333333333333333,
   0.8333333333333333,
   0.83333333333

In [ ]:
[parsed_result['eta_pair'] for parsed_result in parsed_results]

[(0, 29), (0, 59), (0, 90), (29, 59), (29, 90), (59, 90)]

In [120]:
len(parsed_results[0]['etas_third_channel'])

20

In [123]:
parsed_results[0]['etas_third_channel']

[0, 4, 9, 14, 18, 23, 28, 33, 37, 42, 47, 52, 56, 61, 66, 71, 75, 80, 85, 90]

In [89]:
parsed_results[0]['probabilities']

[0.8333333333333333,
 0.8333333333333333,
 0.8337905807041609,
 0.8337905807041609,
 0.8333333333333333,
 0.8333333333333333,
 0.8333333333333333,
 0.8333333333333333,
 0.8333333333333333,
 0.8333333333333333,
 0.7222222222222221,
 0.7222222222222221,
 0.8333332636416139,
 0.8333332636416139,
 0.8333333333333333,
 0.8333333333333333,
 0.5,
 0.5,
 0.796245491032871,
 0.796245491032871,
 0.8331809175430573,
 0.8331809175430573,
 0.8209876543209875,
 0.8209876543209875,
 0.8333333333333333,
 0.8333333333333333,
 0.7222222222222221,
 0.7222222222222221,
 0.7962962937151215,
 0.7962962937151215,
 0.7962962962962963,
 0.7962962962962963,
 0.8333333333333333,
 0.8333333333333333,
 0.7962962962962963,
 0.7962962962962963,
 0.7222222222222221,
 0.7222222222222221,
 0.7222222222222221,
 0.7222222222222221]

In [44]:
one_eta_pair_result = {
        'probabilities': [],
        'etas_third_channel': [],
        'eta_pair': (0,0)
    }

In [45]:
[one_eta_pair_result]*6

[{'probabilities': [], 'etas_third_channel': [], 'eta_pair': (0, 0)},
 {'probabilities': [], 'etas_third_channel': [], 'eta_pair': (0, 0)},
 {'probabilities': [], 'etas_third_channel': [], 'eta_pair': (0, 0)},
 {'probabilities': [], 'etas_third_channel': [], 'eta_pair': (0, 0)},
 {'probabilities': [], 'etas_third_channel': [], 'eta_pair': (0, 0)},
 {'probabilities': [], 'etas_third_channel': [], 'eta_pair': (0, 0)}]